In [ ]:
#https://tw.stock.yahoo.com/quote/6592

In [1]:
import requests
from bs4 import BeautifulSoup

#類別名稱
class Stock:  
    #建構式     (#參數1, 參數2)
    def __init__(self, *stock_num):   #用*變成一個turple
        self.stock_num = stock_num    #屬性(Attribute)
        
    #方法, 爬取
    def scrape(self): 
        result = list() # 最終結果 使用List把所有結果打包起來
        
        
        for stock_num in self.stock_num :
            
            response = requests.get(f"https://tw.stock.yahoo.com/quote/{stock_num}")
            soup = BeautifulSoup( response.text, 'lxml') #指定 lxml 作為解析器             

            stock_name = soup.find( "h1",{ "class" : "C($c-link-text) Fw(b) Fz(24px) Mend(8px)"}).getText() #不想要html標籤
            stock_data = soup.find( "span",{ "class": "C(#6e7780) Fz(14px) As(c)"}).getText().replace('資料時間：', '')
            market_data = stock_data[0:10]  #日期                                             #替換字型
            market_time = stock_data[11:16] #時間

            day_data = soup.find( "ul",{ "class": "D(f) Fld(c) Flw(w) H(192px) Mx(-16px)"}) #當日行情資料
            iteams = day_data.find_all( "li",{"class": "price-detail-item H(32px) Mx(16px) D(f) Jc(sb) Ai(c) Bxz(bb) Px(0px) Py(4px) Bdbs(s) Bdbc($bd-primary-divider) Bdbw(1px)"})
                                

            data = tuple( iteam.find_all("span")[1].getText() for iteam in iteams )

            result.append( ( market_data, stock_name, market_time) + data)
        return result
    
stock = Stock("6592", "2454","2330","1301","2201","2206") #想知道的股票代碼
print( stock.scrape() )

[('2023/11/24', '和潤企業', '13:30', '128.5', '129.0', '129.5', '128.0', '128.5', '0.477', '128.5', '0.00%', '0.00', '371', '377', '1.17%'), ('2023/11/24', '聯發科', '14:30', '945', '935', '950', '932', '943', '42.37', '930', '1.61%', '15.00', '4,493', '5,181', '1.94%'), ('2023/11/24', '台積電', '14:30', '575', '577', '578', '574', '575', '57.73', '578', '0.52%', '3.00', '10,034', '14,459', '0.69%'), ('2023/11/24', '台塑', '14:30', '81.6', '81.5', '81.9', '81.2', '81.5', '3.45', '80.9', '0.87%', '0.70', '4,227', '2,599', '0.87%'), ('2023/11/24', '裕隆', '14:30', '78.5', '78.2', '79.0', '77.8', '78.4', '2.69', '77.8', '0.90%', '0.70', '3,425', '4,450', '1.54%'), ('2023/11/24', '三陽工業', '14:30', '74.7', '74.9', '75.5', '74.4', '74.7', '4.42', '74.5', '0.27%', '0.20', '5,909', '4,440', '1.48%')]


In [2]:
import pandas as pd

# 股票的數值
data = stock.scrape()
                #(數據 , colums(列) 數據上面的名稱標題)
df = pd.DataFrame(data, columns=["日期", "股票名稱", "時間", "成交", "開盤", "最高", "最低", "均價","成交值(億)","昨天收盤","漲跌幅","漲跌","總量","昨量","振幅"])

# 利用 pandas 的 DataFrame 來寫回 excel 
# to_excel寫回去("excel的路徑",index = False不要顯示索引值)
df.to_excel("股票專題.xlsx", index=False)

In [3]:
import xlwings as xw 
wb = xw.Book("股票專題.xlsx")
sheet = wb.sheets["Sheet1"]

#更改一下標題顏色
sheet.range("A1:O1").color = (255, 255, 128)